In [157]:
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import r2_score

In [193]:
df = pd.read_csv('housing.csv')

Elimina los máximos

## Crea un modelo de regresión lineal simple usando el ingreso y calcula su R2

## Crea un modelo de regresión lineal simple usando la edad de las casas

## Crea un modelo de regresión lineal multiple usando las variables anteriores

Escribe la ecuación del modelo

In [1]:
# Interepto


In [2]:
# Coeficientes


In [3]:
# Orden


$$Precio = \beta_0 + \beta_1 * x_1 + \beta_2* x_2$$

$$Precio = -10189 + 1744 * edad + 43169* ingreso$$

### Agrega la variable de households

Escribe la ecuación del modelo

### Agrega todas

In [4]:
### Verific los NA's


In [5]:
# Quita NA's 


In [6]:
# Convertir a one hot encoding 


### Visualizando el modelo, calibracion

### Compara la calibración de todos los modelos anteriores